### **0. Подготовка обучающей и тестовой выборки**

In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
%matplotlib inline 
sns.set(style="ticks")

In [2]:
# Будем использовать только обучающую выборку
data = pd.read_csv('data/train.csv', sep=",")

In [3]:
X = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]  # Пример признаков
y = data['Survived']  # Целевая переменная (1 - выжил, 0 - не выжил)
# Разделение на train и test (сначала split, потом обработка, чтобы избежать data leakage)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
data = data.dropna(subset=['Embarked'])
data = data.drop('Cabin', axis=1)

In [5]:
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Заполняем пропуски медианой
    ('scaler', StandardScaler())  # Масштабируем
])

In [6]:
categorical_features = ['Sex', 'Pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Заполняем пропуски модой
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Кодируем категории
])

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [8]:
# Применяем преобразования
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [9]:
X_train_processed

array([[-1.91971935, -0.47416141,  1.99885349, ...,  1.        ,
         0.        ,  0.        ],
       [-0.0772525 , -0.47416141, -0.47932706, ...,  0.        ,
         0.        ,  1.        ],
       [-2.15002771,  0.34868694,  0.75976322, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.92075038,  1.17153529, -0.47932706, ...,  0.        ,
         0.        ,  1.        ],
       [-1.15202483,  0.34868694,  1.99885349, ...,  1.        ,
         0.        ,  0.        ],
       [-0.61463866, -0.47416141,  0.75976322, ...,  1.        ,
         0.        ,  0.        ]])

### **1. Обучение ансамблевых моделей**

#### **1.1 Бэггинг**

множество одинаковых моделей обучаются на случайных подвыборках, итог - голосование

In [ ]:
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier(n_estimators=50, random_state=42)
bagging.fit(X_train_processed, y_train)
y_pred_bagging = bagging.predict(X_test_processed)
print("Bagging Accuracy:", accuracy_score(y_test, y_pred_bagging))

Bagging Accuracy: 0.8059701492537313


#### **1.2 Случайный лес**

Как бэггинг, только случайна не только обучающая подвыборка, но и подмножество обучающих признаков

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, random_state=42)
rf.fit(X_train_processed, y_train)
y_pred_rf = rf.predict(X_test_processed)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest Accuracy: 0.8134328358208955


#### **1.3 AdaBoost**

Модели обучаются последовательно, на каждой итерации увеличивается вес объектов, предсказанных неверно

In [16]:
from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier(n_estimators=50, random_state=42)
adaboost.fit(X_train_processed, y_train)
y_pred_adaboost = adaboost.predict(X_test_processed)
print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_adaboost))

AdaBoost Accuracy: 0.8171641791044776


#### **1.4 Градиентный бустинг**

Каждая новая модель обучается на остатках ошибок предидущей, вместо изменения весов оптимизируется функция потерь

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=50, random_state=42)
gb.fit(X_train_processed, y_train)
y_pred_gb = gb.predict(X_test_processed)
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))

Gradient Boosting Accuracy: 0.8208955223880597


### **2. Сравнение моделей**

In [18]:
print("Bagging Accuracy:", accuracy_score(y_test, y_pred_bagging))
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_adaboost))
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))

Bagging Accuracy: 0.8059701492537313
Random Forest Accuracy: 0.8134328358208955
AdaBoost Accuracy: 0.8171641791044776
Gradient Boosting Accuracy: 0.8208955223880597
